In [52]:
# Import Packages

import pandas as pd
import numpy as np
import os

# Data Processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split

In [2]:
### read in data

beer_dat = pd.read_csv(r"C:\Users\Angus\Documents\UTS MDSI\Advanced DSI\Projects\advdsi_at2\data\raw\beer_reviews.csv")

In [4]:
beer_dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   brewery_id          1586614 non-null  int64  
 1   brewery_name        1586599 non-null  object 
 2   review_time         1586614 non-null  int64  
 3   review_overall      1586614 non-null  float64
 4   review_aroma        1586614 non-null  float64
 5   review_appearance   1586614 non-null  float64
 6   review_profilename  1586266 non-null  object 
 7   beer_style          1586614 non-null  object 
 8   review_palate       1586614 non-null  float64
 9   review_taste        1586614 non-null  float64
 10  beer_name           1586614 non-null  object 
 11  beer_abv            1518829 non-null  float64
 12  beer_beerid         1586614 non-null  int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


In [98]:
# Divide up data
num_cols = ['review_aroma','review_appearance','review_palate','review_taste']

cat_cols = ['brewery_id']
y_cat_cols = ['beer_style']
key_feat = ['brewery_id','review_aroma','review_appearance','review_palate','review_taste','beer_style']

df_cleaned = beer_dat[key_feat]

In [101]:
X_cat_y.beer_style.unique()
X.shape

(1586614, 6)

In [99]:
#Instantiate SC
sc = StandardScaler()
#Scale numerics
df_cleaned[num_cols] = sc.fit_transform(df_cleaned[num_cols])


#scale Cat
X_cat = df_cleaned[cat_cols].astype('category')
df_cleaned.drop(cat_cols, axis=1, inplace=True)
ode = OrdinalEncoder()
X_cat_y = pd.DataFrame(ode.fit_transform(df_cleaned[y_cat_cols]))
X_cat_y.columns = y_cat_cols
df_cleaned.drop(y_cat_cols, axis=1, inplace=True)
X_cat_cols = pd.concat([X_cat, X_cat_y ], axis=1)

#recombine as X
X = pd.concat([df_cleaned, X_cat_cols ], axis=1)

<ipython-input-99-027f1fbe177f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[num_cols] = sc.fit_transform(df_cleaned[num_cols])
c:\users\angus\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
c:\users\angus\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [66]:
### Train test split

train_data_target = X.pop('beer_style')

train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(X, train_data_target, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio)) 


In [81]:
current = os.getcwd()

os.chdir(r"C:\Users\Angus\Documents\UTS MDSI\Advanced DSI\Projects\advdsi_at2\src\models")
from pytorch import PytorchDataset
os.chdir(r"C:\Users\Angus\Documents\UTS MDSI\Advanced DSI\Projects\advdsi_at2\notebooks")

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
class PytorchDataset(Dataset):

    def __init__(self, X, y):
        self.X_tensor = self.to_tensor(X)
        self.y_tensor = self.to_tensor(y)

    def __getitem__(self, index):
        return self.X_tensor[index], self.y_tensor[index]

    def __len__(self):
        return len(self.X_tensor)

    def to_tensor(self, data):
        return torch.Tensor(np.array(data))

In [67]:
# Generate Pytorch sets
train_dataset = PytorchDataset(X=x_train, y=y_train)
val_dataset = PytorchDataset(X=x_val, y=y_val)
test_dataset = PytorchDataset(X=x_test, y=y_test)

In [90]:
current = os.getcwd()

os.chdir(r"C:\Users\Angus\Documents\UTS MDSI\Advanced DSI\Projects\advdsi_at2\src\models")
from null import NullModel
from performance import print_class_perf
from MClasspytorch import PytorchMultiClass, train_classification, test_classification
os.chdir(r"C:\Users\Angus\Documents\UTS MDSI\Advanced DSI\Projects\advdsi_at2\notebooks")

In [74]:
# fit a baseline model for reference

baseline_model = NullModel(target_type='classification')
y_base = baseline_model.fit_predict(y_train)

#results
print_class_perf(y_base, y_train, set_name='Training', average='weighted')

Accuracy Training: 0.07392207478825062
F1 Training: 0.010176666015692377


In [91]:
# Train Multiclass
model = PytorchMultiClass(x_train.shape[1])

In [92]:
#Set device no GPU
device = torch.device('cpu')

In [93]:
print(model)

PytorchMultiClass(
  (layer_1): Linear(in_features=5, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)


In [94]:
# Set criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer and Learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [95]:
N_EPOCHS = 50
BATCH_SIZE = 32

In [96]:
for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

IndexError: Target 44 is out of bounds.